In [ ]:
import pandas as pd

import sys
import os
sys.path.append(os.path.abspath('../code'))

import config
l = 0
d = {}
filename = config.hny["frmpapname"]

# Pay attention to zie ook

with open(f"data/hny/{filename}", encoding="ISO-8859-1") as fileobject:
    for line in fileobject:
        l += 1
        if config.hny["sep"] not in line:
            continue
        if l >= config.hny["startline"]:
            word, definition = line.split(config.hny["sep"], 1)
            # word, _, definition = line.rpartition(config.hny["sep"])
            word = word.strip()[1:]
            definition = definition.strip()

            if word in config.hny["correction"]:
                definition = config.hny["correction"][word]
            d[word] = definition
df = pd.DataFrame.from_dict(d, orient="index", columns=["nl"])
df.index.name = "pap"
df = df.reset_index()
# print(df)

In [ ]:
import re

def extractFirstWord(row, lan):
    raw_pap = row["pap"]
    raw_nl = row["nl"]
    raw = row[lan]

    # Remove tags
    without_tags = re.sub('[\(\[{<].*?[\)\]}>]', "", raw)

    # Remove numbers
    without_num = re.sub('[0-9][\.:]', "", without_tags)

    without_num = without_num.strip()

    # Only do match if there is a special character, facilitates match until reaching the character
    if re.findall(r'[;\.,!\(]+',without_num):
        match = re.findall(r'^(.*?)[;\.,!\(].*',without_num)        

        # Select first match
        match = match[0]
    else:
        match = without_num
    
    # Remove stray characters
    match = re.sub('[\(\)\]]', "", match)

    # Remove spaces and make everything lowercase
    match = match.strip().lower()

    # Check if there are characters with non letters
    if not all(x.isalpha() or x.isspace() for x in match):
        print(f"Without tags/num was:{without_num}")
        print(f"pap:{raw_pap}|cleaned:{match}|nl:{raw_nl} is not letters")
    
    # if len(match) > 21:
    #     print(f"long word:{match}|raw:{raw}")

    return match

# Drop empty rows
print(f"Length before dropping n/a: {len(df)}")
df['pap'].replace('', pd.np.nan, inplace=True)
df.dropna(subset=["pap"], inplace=True)
print(f"Length after dropping n/a: {len(df)}")

# Process simple version of words
df["nl-simple"] = df.apply(lambda row: extractFirstWord(row, "nl"), axis=1)
df["pap-simple"] = df.apply(lambda row: extractFirstWord(row, "pap"), axis=1)
df["type"] = "word"
df.set_index("pap-simple", inplace=True)

# Save to file
df.to_csv("data/hny/pap-nl.csv")

# Extract more from other way around